# MQTT IoT Subscriber V1.0.0

Horcht auf MQTT Topics

- - -

Als erstes importieren wir die benötigten Libraries

In [ ]:
import os
import uuid
import json
import requests
import paho.mqtt.client as mqtt
from flask import Flask, request, jsonify, render_template_string

#  unique UUID per installation
%run ~/work/uuid.py

Dann folgen die Funktionen um
* sich mit dem MQTT Broker zu verbinden

In [ ]:
app = Flask(__name__)

# Environment variables for configuration
MQTT_BROKER = os.getenv('MQTT_BROKER', 'cloud.tbz.ch')
MQTT_PORT = int(os.getenv('MQTT_PORT', 11883))

# Create a list of topics with UUID
desired_topics = ["iot-alert", "iot-pipe", "iot-consumer"]
MQTT_TOPICS = [f"{topic}/{UUID}" for topic in desired_topics]

# List to store received messages
received_messages = []

# MQTT client setup
client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    print(f"Connected to MQTT broker with result code {rc}")
    for topic in MQTT_TOPICS:
        client.subscribe(topic)
        print(f"Subscribed to topic: {topic}")

def on_message(client, userdata, msg):
    payload = msg.payload.decode('utf-8')
    print(f"Received message on {msg.topic}: {payload}")
    event = {'topic': msg.topic, 'message': payload}
    received_messages.insert(0, event)  # Insert at the beginning to keep the latest message on top

client.on_connect = on_connect
client.on_message = on_message

client.connect(MQTT_BROKER, MQTT_PORT, 60)

Eine Weboberfläche vervollständigen den Microservice

In [ ]:
# HTML template for displaying messages
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Received MQTT Messages</title>
    <style>
        body { font-family: Arial, sans-serif; }
        h1 { color: #333; }
        ul { list-style-type: none; padding: 0; }
        li { padding: 10px; margin: 5px 0; background: #f9f9f9; border: 1px solid #ddd; }
        li strong { color: #555; }
    </style>
</head>
<body>
    <h1>Received MQTT Messages</h1>
    <ul>
    {% for message in messages %}
        <li><strong>{{ message.topic }}</strong>: {{ message.message }}</li>
    {% endfor %}
    </ul>
</body>
</html>
"""

# Endpoint to display received messages as HTML
@app.route('/messages', methods=['GET'])
def display_messages():
    return render_template_string(HTML_TEMPLATE, messages=received_messages)

Zum Schluss starten wir den Microservice auf Port 8080/messages und geben vorher den URL aus:

In [ ]:
! echo "http://$(cat ~/work/server-ip):8080/messages"

In [ ]:
if __name__ == '__main__':
    # Start the MQTT client loop in a separate thread
    client.loop_start()

    port = int(os.getenv('PORT', 8080))
    app.run(host='0.0.0.0', port=port)